# Sea level input

Produce some offline {T1, T2, T3} scenarios for Lennart to run in the sea level rise module.

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ['ssp119', 'ssp245', 'ssp585']

In [ ]:
df_solar = pd.read_csv(
    "../data/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/volcanic_ERF_monthly_-950001-201912.csv"
)

In [ ]:
solar_forcing = np.zeros(751)
volcanic_forcing = np.zeros(751)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]
solar_forcing[:551] = df_solar["erf"].loc[1750:2300].values

trend_shape = np.ones(751)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_configs = pd.read_csv('../data/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
valid_all = df_configs.index

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2500, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties(filename='../data/species_configs_properties_updated.csv')
f.define_species(species, properties)
f.allocate()

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

In [ ]:
df_emis = pd.read_csv(rcmip_emissions_file)

In [ ]:
gfed_sectors = [
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
]

for scenario in scenarios:
    f.emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:750][:, None]

In [ ]:
# from FaIR name to RCMIP name. NOx is excluded from this list as we don't want to pick up the uncorrected value.
# aviation NOx is a CEDS product so it's fine
species_mapping = {'BC': 'Emissions|BC',
 'CCl4': 'Emissions|Montreal Gases|CCl4',
 'CFC-11': 'Emissions|Montreal Gases|CFC|CFC11',
 'CFC-113': 'Emissions|Montreal Gases|CFC|CFC113',
 'CFC-114': 'Emissions|Montreal Gases|CFC|CFC114',
 'CFC-115': 'Emissions|Montreal Gases|CFC|CFC115',
 'CFC-12': 'Emissions|Montreal Gases|CFC|CFC12',
 'CH2Cl2': 'Emissions|Montreal Gases|CH2Cl2',
 'CH3Br': 'Emissions|Montreal Gases|CH3Br',
 'CH3CCl3': 'Emissions|Montreal Gases|CH3CCl3',
 'CH3Cl': 'Emissions|Montreal Gases|CH3Cl',
 'CH4': 'Emissions|CH4',
 'CHCl3': 'Emissions|Montreal Gases|CHCl3',
 'CO': 'Emissions|CO',
 'CO2 AFOLU': 'Emissions|CO2|MAGICC AFOLU',
 'CO2 FFI': 'Emissions|CO2|MAGICC Fossil and Industrial',
 'HCFC-141b': 'Emissions|Montreal Gases|HCFC141b',
 'HCFC-142b': 'Emissions|Montreal Gases|HCFC142b',
 'HCFC-22': 'Emissions|Montreal Gases|HCFC22',
 'HFC-125': 'Emissions|F-Gases|HFC|HFC125',
 'HFC-134a': 'Emissions|F-Gases|HFC|HFC134a',
 'HFC-143a': 'Emissions|F-Gases|HFC|HFC143a',
 'HFC-152a': 'Emissions|F-Gases|HFC|HFC152a',
 'HFC-227ea': 'Emissions|F-Gases|HFC|HFC227ea',
 'HFC-23': 'Emissions|F-Gases|HFC|HFC23',
 'HFC-236fa': 'Emissions|F-Gases|HFC|HFC236fa',
 'HFC-245fa': 'Emissions|F-Gases|HFC|HFC245fa',
 'HFC-32': 'Emissions|F-Gases|HFC|HFC32',
 'HFC-365mfc': 'Emissions|F-Gases|HFC|HFC365mfc',
 'HFC-4310mee': 'Emissions|F-Gases|HFC|HFC4310mee',
 'Halon-1202': 'Emissions|Montreal Gases|Halon1202',
 'Halon-1211': 'Emissions|Montreal Gases|Halon1211',
 'Halon-1301': 'Emissions|Montreal Gases|Halon1301',
 'Halon-2402': 'Emissions|Montreal Gases|Halon2402',
 'N2O': 'Emissions|N2O',
 'NF3': 'Emissions|F-Gases|NF3',
 'NH3': 'Emissions|NH3',
 'NOx aviation': 'Emissions|NOx|MAGICC Fossil and Industrial|Aircraft',
 'OC': 'Emissions|OC',
 'C2F6': 'Emissions|F-Gases|PFC|C2F6',
 'C3F8': 'Emissions|F-Gases|PFC|C3F8',
 'C4F10': 'Emissions|F-Gases|PFC|C4F10',
 'C5F12': 'Emissions|F-Gases|PFC|C5F12',
 'C6F14': 'Emissions|F-Gases|PFC|C6F14',
 'C7F16': 'Emissions|F-Gases|PFC|C7F16',
 'C8F18': 'Emissions|F-Gases|PFC|C8F18',
 'CF4': 'Emissions|F-Gases|PFC|CF4',
 'c-C4F8': 'Emissions|F-Gases|PFC|cC4F8',
 'SF6': 'Emissions|F-Gases|SF6',
 'SO2F2': 'Emissions|F-Gases|SO2F2',
 'Sulfur': 'Emissions|Sulfur',
 'VOC': 'Emissions|VOC'}

In [ ]:
# change units for some emitted species
unit_convert = {specie: 1 for specie in species}
unit_convert['CO2 AFOLU'] = 0.001
unit_convert['CO2 FFI'] = 0.001
unit_convert['N2O'] = 0.001

In [ ]:
for scenario in scenarios:
    for specie in species_mapping:
        f.emissions.loc[dict(specie=specie, scenario=scenario)] = (
            df_emis.loc[
                (df_emis["Scenario"] == scenario)
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"] == species_mapping[specie]),
                "1750":"2500",
            ]
            .interpolate(axis=1)
            .values.squeeze()
            * unit_convert[specie]
        )[:750][:, None]

In [ ]:
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
)

In [ ]:
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

In [ ]:
f.fill_species_configs('../data/species_configs_properties_updated.csv')

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs["rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(f.species_configs["aci_shape"], df_configs["shape Sulfur"].values.squeeze(), specie="Sulfur")
fill(f.species_configs["aci_shape"], df_configs["shape BC"].values.squeeze(), specie="BC")
fill(f.species_configs["aci_shape"], df_configs["shape OC"].values.squeeze(), specie="OC")

# aerosol direct
for specie in ["BC", "CH4", "N2O", "NH3", "NOx", "OC", "Sulfur", "VOC", "Equivalent effective stratospheric chlorine"]:
    fill(f.species_configs["erfari_radiative_efficiency"], df_configs[f"ari {specie}"], specie=specie)

# forcing scaling
for specie in ["CO2", "CH4", "N2O", "Stratospheric water vapour", "Contrails", "Light absorbing particles on snow and ice", "Land use"]:
    fill(f.species_configs["forcing_scale"], df_configs[f"scale {specie}"].values.squeeze(), specie=specie)

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(f.species_configs["forcing_scale"], df_configs["scale minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(f.species_configs["ozone_radiative_efficiency"], df_configs[f"o3 {specie}"], specie=specie)

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs["baseline_concentration"], df_configs["co2_concentration_1750"].values.squeeze(), specie="CO2")

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
for scenario in scenarios:
    pl.fill_between(
        f.timebounds,
        f.temperature.loc[dict(layer=0, scenario=scenario)].quantile(0.05, dim='config'),
        f.temperature.loc[dict(layer=0, scenario=scenario)].quantile(0.95, dim='config'),
        alpha=0.3,
        lw=0
    )
pl.plot(f.timebounds, f.temperature.loc[dict(layer=0)].median(dim='config'), label=scenarios)
pl.ylabel('K relative to pre-industrial')
pl.title('Temperature anomaly')
pl.legend(loc='upper left')
pl.xlim(1750, 2500)

In [ ]:
for scenario in scenarios:
    pl.fill_between(
        f.timebounds,
        f.temperature.loc[dict(layer=0, scenario=scenario)][:,:20].quantile(0.05, dim='config'),
        f.temperature.loc[dict(layer=0, scenario=scenario)][:,:20].quantile(0.95, dim='config'),
        alpha=0.3,
        lw=0
    )
pl.plot(f.timebounds, f.temperature.loc[dict(layer=0)][:,:20].median(dim='config'), label=scenarios)
pl.ylabel('K relative to pre-industrial')
pl.title('Temperature anomaly')
pl.legend(loc='upper left')
pl.xlim(1750, 2500)

In [ ]:
os.makedirs('../output/', exist_ok=True)

In [ ]:
f.ocean_heat_content_change

In [ ]:
f.temperature

In [ ]:
ds = xr.Dataset(
    data_vars={
        'temperature': (['timebounds', 'scenario', 'config', 'layer'], f.temperature.data),
        'ocean_heat_content': (['timebounds', 'scenario', 'config'], f.ocean_heat_content_change.data),
    },
    coords={
        'timebounds': f.timebounds,
        'scenario': f.scenarios,
        'config': f.configs,
        'layer': f.layers
    }
)

In [ ]:
ds

In [ ]:
ds.to_netcdf('../output/fair-2.1.0_calibration-1.1.0.nc')